## Convolution Filter로 Edge Detection

1. **Grayscale 변환**:
    - 이미지는 단일 채널로 변환하여 계산량을 줄이고, 엣지 검출에 집중합니다.
2. **Convolution Filter 정의**:
    - **Horizontal Filter**: 가로 방향 에지를 검출.
    - **Vertical Filter**: 세로 방향 에지를 검출.
3. **Convolution 연산**:
    - 이미지의 각 픽셀에 대해 필터를 슬라이딩하며 지역값(Region)을 필터와 곱합니다.
    - 결과값을 합산하여 새로운 픽셀 값을 계산합니다.
4. **값 클리핑**:
    - 계산 결과가 0~255 범위를 벗어나지 않도록 제한.
5. **결과 시각화**:
    - 원본 이미지, 가로 에지, 세로 에지를 함께 출력.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# 샘플 이미지 로드 및 변환
image = Image.open('shapes.png').convert('L')  # Grayscale로 변환
image_array = np.array(image)

# 🧩 [필터 만들기] 사진에서 '모서리(경계)'를 찾는 작은 칸(3x3) 레고 조각들이에요!
#    이 조각을 사진 위에서 굴리면, 가로/세로/대각선 방향의 모서리가 반짝 보여요 ✨

horizontal_filter = np.array([[-1, -1, -1],   # ➖ 가로선을 찾는 필터 (위에서 아래로 밝기 변화를 느낌)
                               [ 0,  0,  0],
                               [ 1,  1,  1]])

vertical_filter = np.array([[-1,  0,  1],     # ⬅️➡️ 세로선을 찾는 필터 (왼→오른 쪽 밝기 변화를 느낌)
                             [-1,  0,  1],
                             [-1,  0,  1]])

diagonal_filter = np.array([[-1,  -1,  0],    # 🔀 대각선(↘︎ / ↗︎) 모서리도 잡아보자!
                             [-1,   0,  1],
                             [ 0,   1,  1]])


# ⚙️ [직접 Convolution] 사진의 작은 구역(3x3)과 필터를 곱하고 모두 더해요.
#    그러면 '얼마나 모서리 같은지' 숫자가 나오죠! (값이 클수록 모서리 느낌이 강함)
def apply_filter(image, kernel):
    # 📦 결과 담을 상자 만들기 (처음엔 0으로 가득!)
    filtered_image = np.zeros_like(image)
    height, width = image.shape
    k = kernel.shape[0] // 2  # 🎯 필터의 중심(3x3이면 중간은 1)

    # 🚶🏻 사진을 위에서 아래로, 왼쪽에서 오른쪽으로 한 칸씩 걸어가요
    for i in range(k, height - k):
        for j in range(k, width - k):
            # 🔍 현재 위치 주변 3x3 조각 꺼내기
            region = image[i - k:i + k + 1, j - k:j + k + 1]
            # 🧮 3x3끼리 '서로 곱해서' 모두 더하기 → 모서리 점수!
            filtered_value = np.sum(region * kernel)
            # 🧯 값이 너무 크거나 작지 않게 0~255로 안전벨트 채우기
            filtered_image[i, j] = min(max(filtered_value, 0), 255)

    return filtered_image

# 🧪 [실험] 같은 사진에 가로/세로/대각선 필터를 각각 적용해봐요
horizontal_edges = apply_filter(image_array, horizontal_filter)  # ➖ 가로 모서리 강조
vertical_edges   = apply_filter(image_array, vertical_filter)    # ⬅️➡️ 세로 모서리 강조
diagonal_edges   = apply_filter(image_array, diagonal_filter)    # 🔀 대각선 모서리 강조

# 🎨 [보기] 원본과 필터 결과를 나란히 보여주기
plt.figure(figsize=(20, 10))
plt.subplot(1, 4, 1); plt.title("Original");           plt.imshow(image_array,   cmap='gray'); plt.axis('off')
plt.subplot(1, 4, 2); plt.title("Horizontal Edges");   plt.imshow(horizontal_edges, cmap='gray'); plt.axis('off')
plt.subplot(1, 4, 3); plt.title("Vertical Edges");     plt.imshow(vertical_edges,   cmap='gray'); plt.axis('off')
plt.subplot(1, 4, 4); plt.title("Diagonal Edges");     plt.imshow(diagonal_edges,   cmap='gray'); plt.axis('off')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'shapes.png'

## Hough Transoform을 통한 경계 검출

1. **이미지 로드 및 변환**:
    - `cv2.imread()`를 사용하여 이미지를 읽어옵니다.
    - `cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)`를 통해 이미지를 흑백(그레이스케일)으로 변환합니다.→ Hough Transform은 그레이스케일 이미지에서 에지 정보를 분석하므로 컬러 이미지를 흑백으로 변환해야 합니다.
2. **Canny Edge Detection**:
    - `cv2.Canny()`를 사용하여 에지(Edge)를 검출합니다.
    - 두 개의 임곗값(50, 150)은 약한 에지와 강한 에지의 기준을 설정합니다.
        - 낮은 값(50): 약한 에지를 허용하는 임곗값.
        - 높은 값(150): 강한 에지만 검출하는 임곗값.
    - 에지 검출 결과는 이진 이미지로 반환됩니다.
3. **Hough Transform으로 선 검출**:
    - `cv2.HoughLinesP()`를 통해 이미지에서 선을 검출합니다.
        - `rho`: 거리 간격의 해상도 (1 픽셀).
        - `theta`: 각도 간격의 해상도 (1도 = np.pi/180).
        - `threshold`: 선으로 간주하기 위한 최소 누적값.
        - `minLineLength`: 선으로 간주하기 위한 최소 길이.
        - `maxLineGap`: 두 점 사이의 최대 간격(이 값보다 크면 다른 선으로 간주).
4. **선 그리기**:
    - Hough Transform에서 반환된 `lines` 배열은 검출된 선의 시작점과 끝점을 나타냅니다.
    - `cv2.line()`을 사용하여 각 선을 이미지 위에 표시합니다.
5. **결과 시각화**:
    - `matplotlib`를 사용하여 결과를 시각화합니다.
    - 첫 번째 서브플롯: Canny Edge Detection 결과를 흑백으로 표시.
    - 두 번째 서브플롯: 검출된 선을 표시한 원본 이미지를 컬러로 표시.